# Logic Field Theory Verification Code

This notebook verifies the lattice properties and Born rule for Logic Field Theory (LFT) as described in Appendix F of 'Logic Field Theory: A Foundational Program' (Longmire, 2025). It includes three tasks:

1. **Distributivity Counter-example**: Confirms non-distributivity in a ℂ^4 lattice.
2. **Orthomodularity Stress-test**: Verifies orthomodularity for 1,000 random cases.
3. **Born Rule Tally**: Demonstrates uniform measure yields |ψ_i|^2 probabilities.

## Dependencies
- Python 3.8+
- numpy
- itertools
- collections

Install via:
```bash
pip install numpy
```

## 1. Distributivity Counter-example

Verifies that the lattice fails distributivity: $L[(a \land (b \lor c))] \neq L[(a \land b) \lor (a \land c)]$ in ℂ^4.

In [ ]:
import numpy as np
import itertools

# Basis vectors in C^4
E = np.eye(4)
A = np.column_stack((E[:,0], E[:,1]))  # span{e1,e2}
B = np.column_stack((E[:,0], E[:,2]))  # span{e1,e3}
C = np.column_stack((E[:,2], E[:,3]))  # span{e3,e4}

# Linear algebra helpers
proj = lambda U: U @ np.linalg.inv(U.T.conj() @ U) @ U.T.conj()
span = lambda *cols: np.linalg.qr(np.column_stack(cols))[0]
inter = lambda U,V: span(*[u for u in U.T if np.allclose(proj(V)@u, u)])
join = lambda U,V: span(*(list(U.T)+list(V.T)))

def dim(U):
    return 0 if U.size==0 else U.shape[1]

left = inter(A, join(B,C))
right = join(inter(A,B), inter(A,C))
print(f"dims: left={dim(left)}, right={dim(right)} (non-equal => no distributivity)")

**Expected Output**: `dims: left=2, right=1 (non-equal => no distributivity)`

This confirms the lattice is non-distributive, as the dimensions differ.

## 2. Orthomodularity Stress-test (1,000 trials)

Tests orthomodularity: for $X \subseteq Y$, $Y = X \lor (Y \land X^\perp)$ in ℂ^4 over 1,000 random cases.

In [ ]:
import numpy.random as rng
rng.seed(0)

for _ in range(1000):
    r = lambda k: np.linalg.qr(rng.randn(4,k))[0]
    X = r(rng.randint(1,3))          # 1-2-D subspace
    Y = span(X, r(rng.randint(1,3))) # force X ⊆ Y
    Xc = span(*[v for v in r(4) if np.allclose(X.T.conj()@v,0)])
    test = join(X, inter(Y, Xc))
    assert dim(test)==dim(Y), "orthomodularity failed"
print("orthomodularity passed 1000 random cases")

**Expected Output**: `orthomodularity passed 1000 random cases`

This confirms the lattice is orthomodular.

## 3. Toy Born-rule Tally

Demonstrates that a uniform measure on microstates yields Born rule probabilities |ψ_i|^2 for a toy state.

In [ ]:
amps = np.array([1/2, np.sqrt(3)/2, 0])
amps /= np.linalg.norm(amps)
probs = np.abs(amps)**2                   # [0.25, 0.75, 0]
N = 1000
micro = [i for i,p in enumerate(probs) for _ in range(int(round(p*N)))]
from collections import Counter
freqs = Counter(micro)
print({i: freqs[i]/len(micro) for i in range(3)})

**Expected Output**: `{0: 0.25, 1: 0.75, 2: 0.0}`

This matches the Born rule probabilities |ψ_i|^2, confirming the uniform measure’s validity.

## Citation

If you use this code, please cite:
```bibtex
@article{longmire2025,
  author = {Longmire, J. D.},
  title = {Logic Field Theory: A Foundational Program},
  journal = {arXiv preprint},
  year = {2025},
  url = {https://github.com/jdlongmire/LFT-position-paper}
}
```